# Task 2.2 - Keras layered model: RNN - LSTM model

## 1. Importing Libraries and Data

In [814]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from numpy import unique
from numpy import reshape
from keras.models import Sequential
from keras.layers import Conv1D, Conv2D, Dense, BatchNormalization, Flatten, MaxPooling1D, Dropout, LSTM, Input
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [815]:
# Create path variable for re-use when importing data
path = r'C:\Users\sorca\Documents\3ML-2_2'

In [816]:
# Load climate dataframe
df_climate = pd.read_csv(os.path.join(path,'Dataset-weather-prediction-dataset-processed.csv'), index_col = False)

In [817]:
# Verify data
df_climate.head()

,DATE,MONTH,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,BELGRADE_humidity,BELGRADE_pressure,BELGRADE_global_radiation,BELGRADE_precipitation,BELGRADE_sunshine,BELGRADE_temp_mean,BELGRADE_temp_min,BELGRADE_temp_max,BUDAPEST_cloud_cover,BUDAPEST_humidity,BUDAPEST_pressure,BUDAPEST_global_radiation,BUDAPEST_precipitation,BUDAPEST_sunshine,BUDAPEST_temp_mean,BUDAPEST_temp_min,BUDAPEST_temp_max,DEBILT_cloud_cover,DEBILT_wind_speed,DEBILT_humidity,DEBILT_pressure,DEBILT_global_radiation,DEBILT_precipitation,DEBILT_sunshine,DEBILT_temp_mean,DEBILT_temp_min,DEBILT_temp_max,DUSSELDORF_cloud_cover,DUSSELDORF_wind_speed,DUSSELDORF_humidity,DUSSELDORF_pressure,DUSSELDORF_global_radiation,DUSSELDORF_precipitation,DUSSELDORF_snow_depth,DUSSELDORF_sunshine,DUSSELDORF_temp_mean,DUSSELDORF_temp_min,DUSSELDORF_temp_max,GDANSK_cloud_cover,GDANSK_humidity,GDANSK_precipitation,GDANSK_snow_depth,GDANSK_temp_mean,GDANSK_temp_min,GDANSK_temp_max,HEATHROW_cloud_cover,HEATHROW_humidity,HEATHROW_pressure,HEATHROW_global_radiation,HEATHROW_precipitation,HEATHROW_snow_depth,HEATHROW_sunshine,HEATHROW_temp_mean,HEATHROW_temp_min,HEATHROW_temp_max,KASSEL_wind_speed,KASSEL_humidity,KASSEL_pressure,KASSEL_global_radiation,KASSEL_precipitation,KASSEL_sunshine,KASSEL_temp_mean,KASSEL_temp_min,KASSEL_temp_max,LJUBLJANA_cloud_cover,LJUBLJANA_wind_speed,LJUBLJANA_humidity,LJUBLJANA_pressure,LJUBLJANA_global_radiation,LJUBLJANA_precipitation,LJUBLJANA_sunshine,LJUBLJANA_temp_mean,LJUBLJANA_temp_min,LJUBLJANA_temp_max,MAASTRICHT_cloud_cover,MAASTRICHT_wind_speed,MAASTRICHT_humidity,MAASTRICHT_pressure,MAASTRICHT_global_radiation,MAASTRICHT_precipitation,MAASTRICHT_sunshine,MAASTRICHT_temp_mean,MAASTRICHT_temp_min,MAASTRICHT_temp_max,MADRID_cloud_cover,MADRID_wind_speed,MADRID_humidity,MADRID_pressure,MADRID_global_radiation,MADRID_precipitation,MADRID_sunshine,MADRID_temp_mean,MADRID_temp_min,MADRID_temp_max,MUNCHENB_cloud_cover,MUNCHENB_humidity,MUNCHENB_global_radiation,MUNCHENB_precipitation,MUNCHENB_snow_depth,MUNCHENB_sunshine,MUNCHENB_temp_mean,MUNCHENB_temp_min,MUNCHENB_temp_max,OSLO_cloud_cover,OSLO_wind_speed,OSLO_humidity,OSLO_pressure,OSLO_global_radiation,OSLO_precipitation,OSLO_snow_depth,OSLO_sunshine,OSLO_temp_mean,OSLO_temp_min,OSLO_temp_max,ROMA_cloud_cover,ROMA_wind_speed,ROMA_humidity,ROMA_pressure,ROMA_sunshine,ROMA_temp_mean,SONNBLICK_cloud_cover,SONNBLICK_wind_speed,SONNBLICK_humidity,SONNBLICK_pressure,SONNBLICK_global_radiation,SONNBLICK_precipitation,SONNBLICK_sunshine,SONNBLICK_temp_mean,SONNBLICK_temp_min,SONNBLICK_temp_max,STOCKHOLM_cloud_cover,STOCKHOLM_pressure,STOCKHOLM_global_radiation,STOCKHOLM_precipitation,STOCKHOLM_sunshine,STOCKHOLM_temp_mean,STOCKHOLM_temp_min,STOCKHOLM_temp_max,TOURS_wind_speed,TOURS_humidity,TOURS_pressure,TOURS_global_radiation,TOURS_precipitation,TOURS_temp_mean,TOURS_temp_min,TOURS_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,19600101,1,7,2.1,0.85,1.018,0.32,0.09,0,0.7,6.5,0.8,10.9,1,0.81,1.0195,0.88,0.00,7.0,3.7,-0.9,7.9,4,0.67,1.017,0.44,0.01,2.3,2.4,-0.4,5.1,7,7.7,0.85,1.0032,0.07,0.25,0.0,9.3,7.4,11.0,8,5.4,0.83,1.0161,0.12,0.08,0,0.0,10.0,7.0,11.5,8,0.91,0.00,0,0.8,-0.3,1.6,7,0.91,1.0010,0.13,0.22,0,0.0,10.6,9.4,8.3,2.9,0.82,1.0094,0.28,0.48,1.6,7.9,3.9,9.4,8,1.4,1.00,1.0173,0.20,0.00,0.0,-0.6,-1.9,0.5,7,8.7,0.83,1.0063,0.22,0.32,1.0,9.5,8.5,11.1,6,0.0,0.92,1.0260,0.53,0.0,1.4,7.6,4.4,10.8,5,0.67,0.20,0.10,0,0.0,6.9,1.1,10.4,8,4.0,0.98,0.9978,0.04,1.14,0,0.0,4.9,3.8,5.9,3,2.6,0.73,1.0152,7.1,7.8,4,4.5,0.73,1.0304,0.48,0.01,2.3,-5.9,-8.5,-3.2,5,1.0114,0.05,0.32,0.0,4.2,2.2,4.9,3.8,0.76,1.0169,1.54,0.44,10.0,7.8,12.2,5,0.88,1.0003,0.45,0.34,0,4.7,8.5,6.0,10.9
1,19600102,1,6,2.1,0.84

In [818]:
# Verify shape
df_climate.shape

(22950, 170)

In [819]:
# Load "pleasant" dataframe
df_pleasant = pd.read_csv(os.path.join(path,'Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv'), index_col = False)

In [820]:
# Verify data
df_pleasant.head()

,DATE,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,19600101,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,19600102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,19600103,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,19600104,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,19600105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [821]:
# Verify shape
df_pleasant.shape

(22950, 16)

## 2. Data Wrangling

- Drop DATE and MONTH from observations and DATE from predictions data set.
- Drop 3 weather stations not included in answers.
- Remove 2 types of observations (columns) missing multiple entries for most stations.
- Fill in 3 individual observations assuming nearby stations have similar weather.

- Export dataset as "Cleaned" version. X shape should be (22950, 135) and y shape should be (22950, 15).

In [823]:
# Drop DATE and MONTH from observations dataframe
df_climate.drop(['DATE', 'MONTH'], axis=1, inplace=True)

In [824]:
# Verify data
df_climate.head()

,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,BELGRADE_humidity,BELGRADE_pressure,BELGRADE_global_radiation,BELGRADE_precipitation,BELGRADE_sunshine,BELGRADE_temp_mean,BELGRADE_temp_min,BELGRADE_temp_max,BUDAPEST_cloud_cover,BUDAPEST_humidity,BUDAPEST_pressure,BUDAPEST_global_radiation,BUDAPEST_precipitation,BUDAPEST_sunshine,BUDAPEST_temp_mean,BUDAPEST_temp_min,BUDAPEST_temp_max,DEBILT_cloud_cover,DEBILT_wind_speed,DEBILT_humidity,DEBILT_pressure,DEBILT_global_radiation,DEBILT_precipitation,DEBILT_sunshine,DEBILT_temp_mean,DEBILT_temp_min,DEBILT_temp_max,DUSSELDORF_cloud_cover,DUSSELDORF_wind_speed,DUSSELDORF_humidity,DUSSELDORF_pressure,DUSSELDORF_global_radiation,DUSSELDORF_precipitation,DUSSELDORF_snow_depth,DUSSELDORF_sunshine,DUSSELDORF_temp_mean,DUSSELDORF_temp_min,DUSSELDORF_temp_max,GDANSK_cloud_cover,GDANSK_humidity,GDANSK_precipitation,GDANSK_snow_depth,GDANSK_temp_mean,GDANSK_temp_min,GDANSK_temp_max,HEATHROW_cloud_cover,HEATHROW_humidity,HEATHROW_pressure,HEATHROW_global_radiation,HEATHROW_precipitation,HEATHROW_snow_depth,HEATHROW_sunshine,HEATHROW_temp_mean,HEATHROW_temp_min,HEATHROW_temp_max,KASSEL_wind_speed,KASSEL_humidity,KASSEL_pressure,KASSEL_global_radiation,KASSEL_precipitation,KASSEL_sunshine,KASSEL_temp_mean,KASSEL_temp_min,KASSEL_temp_max,LJUBLJANA_cloud_cover,LJUBLJANA_wind_speed,LJUBLJANA_humidity,LJUBLJANA_pressure,LJUBLJANA_global_radiation,LJUBLJANA_precipitation,LJUBLJANA_sunshine,LJUBLJANA_temp_mean,LJUBLJANA_temp_min,LJUBLJANA_temp_max,MAASTRICHT_cloud_cover,MAASTRICHT_wind_speed,MAASTRICHT_humidity,MAASTRICHT_pressure,MAASTRICHT_global_radiation,MAASTRICHT_precipitation,MAASTRICHT_sunshine,MAASTRICHT_temp_mean,MAASTRICHT_temp_min,MAASTRICHT_temp_max,MADRID_cloud_cover,MADRID_wind_speed,MADRID_humidity,MADRID_pressure,MADRID_global_radiation,MADRID_precipitation,MADRID_sunshine,MADRID_temp_mean,MADRID_temp_min,MADRID_temp_max,MUNCHENB_cloud_cover,MUNCHENB_humidity,MUNCHENB_global_radiation,MUNCHENB_precipitation,MUNCHENB_snow_depth,MUNCHENB_sunshine,MUNCHENB_temp_mean,MUNCHENB_temp_min,MUNCHENB_temp_max,OSLO_cloud_cover,OSLO_wind_speed,OSLO_humidity,OSLO_pressure,OSLO_global_radiation,OSLO_precipitation,OSLO_snow_depth,OSLO_sunshine,OSLO_temp_mean,OSLO_temp_min,OSLO_temp_max,ROMA_cloud_cover,ROMA_wind_speed,ROMA_humidity,ROMA_pressure,ROMA_sunshine,ROMA_temp_mean,SONNBLICK_cloud_cover,SONNBLICK_wind_speed,SONNBLICK_humidity,SONNBLICK_pressure,SONNBLICK_global_radiation,SONNBLICK_precipitation,SONNBLICK_sunshine,SONNBLICK_temp_mean,SONNBLICK_temp_min,SONNBLICK_temp_max,STOCKHOLM_cloud_cover,STOCKHOLM_pressure,STOCKHOLM_global_radiation,STOCKHOLM_precipitation,STOCKHOLM_sunshine,STOCKHOLM_temp_mean,STOCKHOLM_temp_min,STOCKHOLM_temp_max,TOURS_wind_speed,TOURS_humidity,TOURS_pressure,TOURS_global_radiation,TOURS_precipitation,TOURS_temp_mean,TOURS_temp_min,TOURS_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,7,2.1,0.85,1.018,0.32,0.09,0,0.7,6.5,0.8,10.9,1,0.81,1.0195,0.88,0.00,7.0,3.7,-0.9,7.9,4,0.67,1.017,0.44,0.01,2.3,2.4,-0.4,5.1,7,7.7,0.85,1.0032,0.07,0.25,0.0,9.3,7.4,11.0,8,5.4,0.83,1.0161,0.12,0.08,0,0.0,10.0,7.0,11.5,8,0.91,0.00,0,0.8,-0.3,1.6,7,0.91,1.0010,0.13,0.22,0,0.0,10.6,9.4,8.3,2.9,0.82,1.0094,0.28,0.48,1.6,7.9,3.9,9.4,8,1.4,1.00,1.0173,0.20,0.00,0.0,-0.6,-1.9,0.5,7,8.7,0.83,1.0063,0.22,0.32,1.0,9.5,8.5,11.1,6,0.0,0.92,1.0260,0.53,0.0,1.4,7.6,4.4,10.8,5,0.67,0.20,0.10,0,0.0,6.9,1.1,10.4,8,4.0,0.98,0.9978,0.04,1.14,0,0.0,4.9,3.8,5.9,3,2.6,0.73,1.0152,7.1,7.8,4,4.5,0.73,1.0304,0.48,0.01,2.3,-5.9,-8.5,-3.2,5,1.0114,0.05,0.32,0.0,4.2,2.2,4.9,3.8,0.76,1.0169,1.54,0.44,10.0,7.8,12.2,5,0.88,1.0003,0.45,0.34,0,4.7,8.5,6.0,10.9
1,6,2.1,0.84,1.018,0.36,1.05,0,1.1,6.1,3.3,10

In [825]:
# Drop DATE from predictions dataframe
df_pleasant.drop(['DATE'], axis=1, inplace=True)

In [826]:
# Verify data
df_pleasant.head()

,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [827]:
# Drop 3 weather stations not included in answers
df_climate = df_climate.drop(['GDANSK_cloud_cover', 'GDANSK_humidity', 'GDANSK_precipitation', 'GDANSK_snow_depth', 'GDANSK_temp_mean', 'GDANSK_temp_min', 'GDANSK_temp_max',
                        'ROMA_cloud_cover', 'ROMA_wind_speed', 'ROMA_humidity', 'ROMA_pressure', 'ROMA_sunshine', 'ROMA_temp_mean',
                        'TOURS_wind_speed', 'TOURS_humidity', 'TOURS_pressure', 'TOURS_global_radiation', 'TOURS_precipitation', 'TOURS_temp_mean', 'TOURS_temp_min', 'TOURS_temp_max'], axis=1)

In [828]:
# Verify shape
df_climate.shape

(22950, 147)

In [829]:
# Define a list called observation_types
observation_types = ['cloud_cover', 'wind_speed', 'humidity', 'pressure',
                     'global_radiation', 'precipitation', 'snow_depth', 
                     'sunshine', 'temp_mean', 'temp_min', 'temp_max']

In [830]:
# Remove 2 types of observations (columns) missing multiple entries for most stations.

# Create an empty dictionary named station_counts
station_counts = {}

for obs in observation_types:
    # Select columns related to the current observation type
    columns = [col for col in df_climate.columns if col.endswith(obs)]
    
    # Count the number of stations (or the number of columns) for the current observation type
    station_counts[obs] = len(columns)

# Print the count of stations for each observation type
print("Number of stations for each observation type:")
for obs, count in station_counts.items():
    print(f"{obs}: {count} stations")

Number of stations for each observation type:
cloud_cover: 14 stations
wind_speed: 9 stations
humidity: 14 stations
pressure: 14 stations
global_radiation: 15 stations
precipitation: 15 stations
snow_depth: 6 stations
sunshine: 15 stations
temp_mean: 15 stations
temp_min: 15 stations
temp_max: 15 stations


In [831]:
# Will drop the columns ending in wind_speed and snow_depth (have data for 9, and 6, respectively)
columns_to_drop = df_climate.filter(regex='(_wind_speed|_snow_depth)$').columns
columns_to_drop

Index(['BASEL_wind_speed', 'BASEL_snow_depth', 'DEBILT_wind_speed',
       'DUSSELDORF_wind_speed', 'DUSSELDORF_snow_depth', 'HEATHROW_snow_depth',
       'KASSEL_wind_speed', 'LJUBLJANA_wind_speed', 'MAASTRICHT_wind_speed',
       'MADRID_wind_speed', 'MUNCHENB_snow_depth', 'OSLO_wind_speed',
       'OSLO_snow_depth', 'SONNBLICK_wind_speed', 'VALENTIA_snow_depth'],
      dtype='object')

In [832]:
# Drop the columns listed above
df_climate = df_climate.drop(columns=columns_to_drop)

In [833]:
df_climate.shape

(22950, 132)

In [834]:
# We are missing one entry for each the following observations: cloud_cover, humidity, and pressure
# We need to determine which stations are missing

# Create a list of all unique station names in the dataset
all_stations = set([col.split('_')[0] for col in df_climate.columns if '_' in col])
all_stations

{'BASEL',
 'BELGRADE',
 'BUDAPEST',
 'DEBILT',
 'DUSSELDORF',
 'HEATHROW',
 'KASSEL',
 'LJUBLJANA',
 'MAASTRICHT',
 'MADRID',
 'MUNCHENB',
 'OSLO',
 'SONNBLICK',
 'STOCKHOLM',
 'VALENTIA'}

In [835]:
# Create a list with the observation types
observation_types = ['cloud_cover', 'humidity', 'pressure']

# Create a dictionary for the stations which are missing the above observation types
missing_stations_by_observation = {}

for obs in observation_types:
    # Select columns related to the current observation type
    columns = [col for col in df_climate.columns if col.endswith(obs)]
    
    # Extract station names
    station_names = set([col.replace(f'_{obs}', '') for col in columns])
    
    # Identify stations that are in all_stations but missing from the current observation type
    missing_stations = all_stations - station_names
    
    # Store the missing station names in the dictionary
    missing_stations_by_observation[obs] = missing_stations

# Print the missing station names for each observation type
for obs, missing_stations in missing_stations_by_observation.items():
    print(f"\nStations with no {obs}:")
    if missing_stations:
        for station in missing_stations:
            print(station)
    else:
        print("None")


Stations with no cloud_cover:
KASSEL

Stations with no humidity:
STOCKHOLM

Stations with no pressure:
MUNCHENB


In [836]:
# Find the position of HEATHROW_temp_max so that we know where to insert the new column KASSEL_cloud_cover

df_climate.columns.get_loc('HEATHROW_temp_max')

53

In [837]:
# Find the position of STOCKHOLM_cloud_cover so that we know where to insert the new column STOCKHOLM_humidity

df_climate.columns.get_loc('STOCKHOLM_cloud_cover')

115

In [838]:
# Find the position of MUNCHENB_humidity so that we know where to insert the new column MUNCHENB_pressure

df_climate.columns.get_loc('MUNCHENB_humidity')

90

In [839]:
# Insert the new columns into dataframe df_climate at the positions found above.
# The data for the new columns is duplicated from the following existing columns, which are from a similar climate:
# Dusseldorf_cloud_cover (for Kassel_cloud_cover)
# Oslo_humidity (for Stockholm_humidity)
# Basel_pressure (for Munchenb_pressure)

df_climate.insert(54,'KASSEL_cloud_cover', df_climate['DUSSELDORF_cloud_cover'])
df_climate.insert(117, 'STOCKHOLM_humidity', df_climate['OSLO_humidity'])
df_climate.insert(92,'MUNCHENB_pressure',df_climate['BASEL_pressure'])

In [840]:
# Change to display all columns in the df_climate.head()
pd.set_option('display.max_columns', None)
df_climate.head()

,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,BELGRADE_humidity,BELGRADE_pressure,BELGRADE_global_radiation,BELGRADE_precipitation,BELGRADE_sunshine,BELGRADE_temp_mean,BELGRADE_temp_min,BELGRADE_temp_max,BUDAPEST_cloud_cover,BUDAPEST_humidity,BUDAPEST_pressure,BUDAPEST_global_radiation,BUDAPEST_precipitation,BUDAPEST_sunshine,BUDAPEST_temp_mean,BUDAPEST_temp_min,BUDAPEST_temp_max,DEBILT_cloud_cover,DEBILT_humidity,DEBILT_pressure,DEBILT_global_radiation,DEBILT_precipitation,DEBILT_sunshine,DEBILT_temp_mean,DEBILT_temp_min,DEBILT_temp_max,DUSSELDORF_cloud_cover,DUSSELDORF_humidity,DUSSELDORF_pressure,DUSSELDORF_global_radiation,DUSSELDORF_precipitation,DUSSELDORF_sunshine,DUSSELDORF_temp_mean,DUSSELDORF_temp_min,DUSSELDORF_temp_max,HEATHROW_cloud_cover,HEATHROW_humidity,HEATHROW_pressure,HEATHROW_global_radiation,HEATHROW_precipitation,HEATHROW_sunshine,HEATHROW_temp_mean,HEATHROW_temp_min,HEATHROW_temp_max,KASSEL_cloud_cover,KASSEL_humidity,KASSEL_pressure,KASSEL_global_radiation,KASSEL_precipitation,KASSEL_sunshine,KASSEL_temp_mean,KASSEL_temp_min,KASSEL_temp_max,LJUBLJANA_cloud_cover,LJUBLJANA_humidity,LJUBLJANA_pressure,LJUBLJANA_global_radiation,LJUBLJANA_precipitation,LJUBLJANA_sunshine,LJUBLJANA_temp_mean,LJUBLJANA_temp_min,LJUBLJANA_temp_max,MAASTRICHT_cloud_cover,MAASTRICHT_humidity,MAASTRICHT_pressure,MAASTRICHT_global_radiation,MAASTRICHT_precipitation,MAASTRICHT_sunshine,MAASTRICHT_temp_mean,MAASTRICHT_temp_min,MAASTRICHT_temp_max,MADRID_cloud_cover,MADRID_humidity,MADRID_pressure,MADRID_global_radiation,MADRID_precipitation,MADRID_sunshine,MADRID_temp_mean,MADRID_temp_min,MADRID_temp_max,MUNCHENB_cloud_cover,MUNCHENB_humidity,MUNCHENB_pressure,MUNCHENB_global_radiation,MUNCHENB_precipitation,MUNCHENB_sunshine,MUNCHENB_temp_mean,MUNCHENB_temp_min,MUNCHENB_temp_max,OSLO_cloud_cover,OSLO_humidity,OSLO_pressure,OSLO_global_radiation,OSLO_precipitation,OSLO_sunshine,OSLO_temp_mean,OSLO_temp_min,OSLO_temp_max,SONNBLICK_cloud_cover,SONNBLICK_humidity,SONNBLICK_pressure,SONNBLICK_global_radiation,SONNBLICK_precipitation,SONNBLICK_sunshine,SONNBLICK_temp_mean,SONNBLICK_temp_min,SONNBLICK_temp_max,STOCKHOLM_cloud_cover,STOCKHOLM_humidity,STOCKHOLM_pressure,STOCKHOLM_global_radiation,STOCKHOLM_precipitation,STOCKHOLM_sunshine,STOCKHOLM_temp_mean,STOCKHOLM_temp_min,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,7,0.85,1.018,0.32,0.09,0.7,6.5,0.8,10.9,1,0.81,1.0195,0.88,0.00,7.0,3.7,-0.9,7.9,4,0.67,1.017,0.44,0.01,2.3,2.4,-0.4,5.1,7,0.85,1.0032,0.07,0.25,0.0,9.3,7.4,11.0,8,0.83,1.0161,0.12,0.08,0.0,10.0,7.0,11.5,7,0.91,1.0010,0.13,0.22,0.0,10.6,9.4,8.3,8,0.82,1.0094,0.28,0.48,1.6,7.9,3.9,9.4,8,1.00,1.0173,0.20,0.00,0.0,-0.6,-1.9,0.5,7,0.83,1.0063,0.22,0.32,1.0,9.5,8.5,11.1,6,0.92,1.0260,0.53,0.0,1.4,7.6,4.4,10.8,5,0.67,1.018,0.20,0.10,0.0,6.9,1.1,10.4,8,0.98,0.9978,0.04,1.14,0.0,4.9,3.8,5.9,4,0.73,1.0304,0.48,0.01,2.3,-5.9,-8.5,-3.2,5,0.98,1.0114,0.05,0.32,0.0,4.2,2.2,4.9,5,0.88,1.0003,0.45,0.34,4.7,8.5,6.0,10.9
1,6,0.84,1.018,0.36,1.05,1.1,6.1,3.3,10.1,6,0.84,1.0172,0.25,0.00,0.0,2.9,2.2,4.4,4,0.67,1.017,0.18,0.31,0.0,2.3,1.4,3.1,8,0.90,1.0056,0.14,0.06,0.1,7.7,6.4,8.3,8,0.89,1.0161,0.18,0.66,0.5,8.2,7.4,11.0,7,0.98,1.0051,0.13,0.23,0.0,6.1,3.9,10.6,8,0.86,1.0086,0.12,0.27,0.0,7.7,6.8,9.1,6,0.94,1.0173,0.56,0.13,3.2,2.1,-1.3,5.5,8,0.92,1.0062,0.17,1.34,0.4,8.6,7.5,9.9,7,0.86,1.0254,0.46,0.0,0.9,9.8,7.4,12.2,6,0.72,1.018,0.61,0.30,5.1,6.2,4.2,10.2,8,0.62,1.0139,0.04,0.00,0.0,3.4,2.8,4.9,6,0.97,1.0292,0.21,0.61,0.0,-9.5,-10.5,-8.5,5,0.62,1.0114,0.05,0.06,0.0,4.0,3.0,5.0,7,0.91,1.0007,0.25,0.84,0.7,8.9,5.6,12.1
2,8,0.90,1.018,0.18,0.30,0.0,8.5,5.1,9.9,6,0.77,1.0179,0.67,0.00,3.5,3.1,-0.5,6.4,4,0.67,1.017,0.30,0.00,0.6,2.7,1.7,5.3,6,0.92,1.0165,0.28,0.01,

In [841]:
df_climate.columns.tolist()

['BASEL_cloud_cover',
 'BASEL_humidity',
 'BASEL_pressure',
 'BASEL_global_radiation',
 'BASEL_precipitation',
 'BASEL_sunshine',
 'BASEL_temp_mean',
 'BASEL_temp_min',
 'BASEL_temp_max',
 'BELGRADE_cloud_cover',
 'BELGRADE_humidity',
 'BELGRADE_pressure',
 'BELGRADE_global_radiation',
 'BELGRADE_precipitation',
 'BELGRADE_sunshine',
 'BELGRADE_temp_mean',
 'BELGRADE_temp_min',
 'BELGRADE_temp_max',
 'BUDAPEST_cloud_cover',
 'BUDAPEST_humidity',
 'BUDAPEST_pressure',
 'BUDAPEST_global_radiation',
 'BUDAPEST_precipitation',
 'BUDAPEST_sunshine',
 'BUDAPEST_temp_mean',
 'BUDAPEST_temp_min',
 'BUDAPEST_temp_max',
 'DEBILT_cloud_cover',
 'DEBILT_humidity',
 'DEBILT_pressure',
 'DEBILT_global_radiation',
 'DEBILT_precipitation',
 'DEBILT_sunshine',
 'DEBILT_temp_mean',
 'DEBILT_temp_min',
 'DEBILT_temp_max',
 'DUSSELDORF_cloud_cover',
 'DUSSELDORF_humidity',
 'DUSSELDORF_pressure',
 'DUSSELDORF_global_radiation',
 'DUSSELDORF_precipitation',
 'DUSSELDORF_sunshine',
 'DUSSELDORF_temp_mean',


In [842]:
# Check the shape of the observation dataframe (X)
df_climate.shape

(22950, 135)

In [843]:
# Check the shape of the predictions dataframe (y)
df_pleasant.shape

(22950, 15)

The shape of the observations and predictions dataframes are verified to be as expected.

In [845]:
# Export the wrangled data

df_climate.to_csv(os.path.join(path, 'Climate_cleaned.csv'), index=False)
df_pleasant.to_csv(os.path.join(path, 'Pleasant_cleaned.csv'), index=False)

## 4. Reshaping Data for Modeling

In [847]:
# Create arrays X and y from dataframes

X = np.array(df_climate)
y = np.array(df_pleasant)

In [848]:
X

array([[ 7.    ,  0.85  ,  1.018 , ...,  8.5   ,  6.    , 10.9   ],
       [ 6.    ,  0.84  ,  1.018 , ...,  8.9   ,  5.6   , 12.1   ],
       [ 8.    ,  0.9   ,  1.018 , ..., 10.5   ,  8.1   , 12.9   ],
       ...,
       [ 4.    ,  0.76  ,  1.0227, ..., 10.7   ,  7.9   , 13.5   ],
       [ 5.    ,  0.8   ,  1.0212, ..., 10.7   ,  7.9   , 13.5   ],
       [ 5.    ,  0.84  ,  1.0193, ..., 10.7   ,  7.9   , 13.5   ]])

In [849]:
y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [850]:
X = X.reshape(-1,15,9)

In [851]:
X

array([[[  7.    ,   0.85  ,   1.018 , ...,   6.5   ,   0.8   ,
          10.9   ],
        [  1.    ,   0.81  ,   1.0195, ...,   3.7   ,  -0.9   ,
           7.9   ],
        [  4.    ,   0.67  ,   1.017 , ...,   2.4   ,  -0.4   ,
           5.1   ],
        ...,
        [  4.    ,   0.73  ,   1.0304, ...,  -5.9   ,  -8.5   ,
          -3.2   ],
        [  5.    ,   0.98  ,   1.0114, ...,   4.2   ,   2.2   ,
           4.9   ],
        [  5.    ,   0.88  ,   1.0003, ...,   8.5   ,   6.    ,
          10.9   ]],

       [[  6.    ,   0.84  ,   1.018 , ...,   6.1   ,   3.3   ,
          10.1   ],
        [  6.    ,   0.84  ,   1.0172, ...,   2.9   ,   2.2   ,
           4.4   ],
        [  4.    ,   0.67  ,   1.017 , ...,   2.3   ,   1.4   ,
           3.1   ],
        ...,
        [  6.    ,   0.97  ,   1.0292, ...,  -9.5   , -10.5   ,
          -8.5   ],
        [  5.    ,   0.62  ,   1.0114, ...,   4.    ,   3.    ,
           5.    ],
        [  7.    ,   0.91  ,   1.0007, ...,   8.

## 5. Split the Data

In [853]:
# Split data into train and test sets

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

In [854]:
# Verify the shapes of train and test data
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(17212, 15, 9) (17212, 15)
(5738, 15, 9) (5738, 15)


## 6. Create and configure LSTM model

### 6.1 Configuration 1

In [857]:
# epochs = 10
# batch_size = 4
# n_hidden = 4
# activation='tanh'

epochs = 10
batch_size = 4
n_hidden = 4

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
# model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Input(shape=(timesteps, input_dim)))
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu'))
model.add(MaxPooling1D())
model.add(LSTM(n_hidden))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='tanh')) # Don't use relu here!

In [858]:
model.summary()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_10 (Conv1D)              │ (None, 14, 4)          │            76 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_10 (MaxPooling1D) │ (None, 7, 4)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_9 (LSTM)                   │ (None, 4)              │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 4)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 15)             │            75 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 295 (1.15 KB)

 Trainable params: 295 (1.15 KB)

 Non-trainable params: 0 (0.00 B)

In [859]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [860]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/10
4303/4303 - 11s - 3ms/step - accuracy: 0.0388 - loss: 25.7550
Epoch 2/10
4303/4303 - 9s - 2ms/step - accuracy: 0.0260 - loss: 24.6262
Epoch 3/10
4303/4303 - 9s - 2ms/step - accuracy: 0.0246 - loss: 25.1263
Epoch 4/10
4303/4303 - 8s - 2ms/step - accuracy: 0.0236 - loss: 25.4940
Epoch 5/10
4303/4303 - 8s - 2ms/step - accuracy: 0.0133 - loss: 25.9556
Epoch 6/10
4303/4303 - 8s - 2ms/step - accuracy: 0.0092 - loss: 25.0623
Epoch 7/10
4303/4303 - 9s - 2ms/step - accuracy: 0.0074 - loss: 24.5398
Epoch 8/10
4303/4303 - 9s - 2ms/step - accuracy: 0.0025 - loss: 24.3925
Epoch 9/10
4303/4303 - 8s - 2ms/step - accuracy: 0.0033 - loss: 25.3442
Epoch 10/10
4303/4303 - 8s - 2ms/step - accuracy: 0.0066 - loss: 24.8066


In [861]:
# Define list of stations names

stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'}

In [862]:
# Define the Confusion Matrix
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [863]:
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Pred        HEATHROW  LJUBLJANA  MUNCHENB  SONNBLICK
True                                                
BASEL              4       1615      2060          3
BELGRADE           0        921       171          0
BUDAPEST           0        169        45          0
DEBILT             0         75         7          0
DUSSELDORF         0         24         5          0
HEATHROW           0         50        32          0
KASSEL             0          8         3          0
LJUBLJANA          0         24        37          0
MAASTRICHT         0          7         2          0
MADRID             0        170       287          1
MUNCHENB           0          7         1          0
OSLO               0          3         2          0
STOCKHOLM          0          4         0          0
VALENTIA           0          0         1          0


### 6.2. Configuration 2

In [865]:
# epochs = 10
# batch_size = 4
# n_hidden = 4
# activation='sigmoid'

epochs = 10
batch_size = 4
n_hidden = 4

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
# model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Input(shape=(timesteps, input_dim)))
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu'))
model.add(MaxPooling1D())
model.add(LSTM(n_hidden))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='sigmoid')) # Don't use relu here!

In [866]:
model.summary()

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_11 (Conv1D)              │ (None, 14, 4)          │            76 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_11 (MaxPooling1D) │ (None, 7, 4)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_10 (LSTM)                  │ (None, 4)              │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 4)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 15)             │            75 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 295 (1.15 KB)

 Trainable params: 295 (1.15 KB)

 Non-trainable params: 0 (0.00 B)

In [867]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [868]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/10
4303/4303 - 10s - 2ms/step - accuracy: 0.1204 - loss: 8.8471
Epoch 2/10
4303/4303 - 8s - 2ms/step - accuracy: 0.0732 - loss: 9.1862
Epoch 3/10
4303/4303 - 8s - 2ms/step - accuracy: 0.0589 - loss: 9.6697
Epoch 4/10
4303/4303 - 9s - 2ms/step - accuracy: 0.0562 - loss: 10.0766
Epoch 5/10
4303/4303 - 9s - 2ms/step - accuracy: 0.0522 - loss: 10.5126
Epoch 6/10
4303/4303 - 8s - 2ms/step - accuracy: 0.0539 - loss: 10.8451
Epoch 7/10
4303/4303 - 8s - 2ms/step - accuracy: 0.0541 - loss: 11.2392
Epoch 8/10
4303/4303 - 8s - 2ms/step - accuracy: 0.0571 - loss: 11.5093
Epoch 9/10
4303/4303 - 9s - 2ms/step - accuracy: 0.0553 - loss: 11.8352
Epoch 10/10
4303/4303 - 8s - 2ms/step - accuracy: 0.0525 - loss: 12.1751


In [869]:
# Define list of stations names

stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'}

In [870]:
# Define the Confusion Matrix
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [871]:
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Pred        BASEL  BELGRADE  BUDAPEST  MADRID
True                                         
BASEL        1490         1        18    2173
BELGRADE      994         0        14      84
BUDAPEST      210         0         1       3
DEBILT         82         0         0       0
DUSSELDORF     28         0         1       0
HEATHROW       73         0         1       8
KASSEL         11         0         0       0
LJUBLJANA      54         0         0       7
MAASTRICHT      7         0         0       2
MADRID        186         0        13     259
MUNCHENB        6         0         0       2
OSLO            4         0         0       1
STOCKHOLM       4         0         0       0
VALENTIA        1         0         0       0


### 6.3. Configuration 3

In [873]:
# epochs = 10
# batch_size = 4
# n_hidden = 4
# activation='softmax'

epochs = 10
batch_size = 4
n_hidden = 4

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
# model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Input(shape=(timesteps, input_dim)))
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu'))
model.add(MaxPooling1D())
model.add(LSTM(n_hidden))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='softmax')) # Don't use relu here!

In [874]:
model.summary()

Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_12 (Conv1D)              │ (None, 14, 4)          │            76 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_12 (MaxPooling1D) │ (None, 7, 4)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_11 (LSTM)                  │ (None, 4)              │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 4)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 15)             │            75 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 295 (1.15 KB)

 Trainable params: 295 (1.15 KB)

 Non-trainable params: 0 (0.00 B)

In [875]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [876]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/10
4303/4303 - 10s - 2ms/step - accuracy: 0.0850 - loss: 8.9593
Epoch 2/10
4303/4303 - 9s - 2ms/step - accuracy: 0.0935 - loss: 9.3209
Epoch 3/10
4303/4303 - 9s - 2ms/step - accuracy: 0.0618 - loss: 9.9739
Epoch 4/10
4303/4303 - 8s - 2ms/step - accuracy: 0.0632 - loss: 10.5456
Epoch 5/10
4303/4303 - 8s - 2ms/step - accuracy: 0.0618 - loss: 11.2033
Epoch 6/10
4303/4303 - 9s - 2ms/step - accuracy: 0.0623 - loss: 11.7839
Epoch 7/10
4303/4303 - 8s - 2ms/step - accuracy: 0.0623 - loss: 12.3662
Epoch 8/10
4303/4303 - 8s - 2ms/step - accuracy: 0.0633 - loss: 12.8917
Epoch 9/10
4303/4303 - 9s - 2ms/step - accuracy: 0.0630 - loss: 13.3841
Epoch 10/10
4303/4303 - 9s - 2ms/step - accuracy: 0.0620 - loss: 13.8421


In [877]:
# Define list of stations names

stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'}

In [878]:
# Define the Confusion Matrix
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [879]:
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Pred        MADRID
True              
BASEL         3682
BELGRADE      1092
BUDAPEST       214
DEBILT          82
DUSSELDORF      29
HEATHROW        82
KASSEL          11
LJUBLJANA       61
MAASTRICHT       9
MADRID         458
MUNCHENB         8
OSLO             5
STOCKHOLM        4
VALENTIA         1


### 6.4. Configuration 4

In [941]:
# epochs = 20
# batch_size = 4
# n_hidden = 4
# activation='sigmoid'

epochs = 20
batch_size = 4
n_hidden = 4

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
# model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Input(shape=(timesteps, input_dim)))
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu'))
model.add(MaxPooling1D())
model.add(LSTM(n_hidden))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='sigmoid')) # Don't use relu here!

In [950]:
model.summary()

Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_13 (Conv1D)              │ (None, 14, 4)          │            76 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_13 (MaxPooling1D) │ (None, 7, 4)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_12 (LSTM)                  │ (None, 4)              │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 4)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 15)             │            75 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 887 (3.47 KB)

 Trainable params: 295 (1.15 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 592 (2.32 KB)

In [952]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [954]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/20
4303/4303 - 11s - 2ms/step - accuracy: 0.0616 - loss: 19.1032
Epoch 2/20
4303/4303 - 8s - 2ms/step - accuracy: 0.0616 - loss: 19.5273
Epoch 3/20
4303/4303 - 8s - 2ms/step - accuracy: 0.0620 - loss: 20.0759
Epoch 4/20
4303/4303 - 9s - 2ms/step - accuracy: 0.0619 - loss: 20.3875
Epoch 5/20
4303/4303 - 9s - 2ms/step - accuracy: 0.0635 - loss: 21.0052
Epoch 6/20
4303/4303 - 9s - 2ms/step - accuracy: 0.0633 - loss: 21.3493
Epoch 7/20
4303/4303 - 8s - 2ms/step - accuracy: 0.0622 - loss: 21.7722
Epoch 8/20
4303/4303 - 9s - 2ms/step - accuracy: 0.0634 - loss: 22.2860
Epoch 9/20
4303/4303 - 9s - 2ms/step - accuracy: 0.0629 - loss: 22.7213
Epoch 10/20
4303/4303 - 8s - 2ms/step - accuracy: 0.0623 - loss: 23.0730
Epoch 11/20
4303/4303 - 9s - 2ms/step - accuracy: 0.0632 - loss: 23.4954
Epoch 12/20
4303/4303 - 9s - 2ms/step - accuracy: 0.0628 - loss: 23.9699
Epoch 13/20
4303/4303 - 9s - 2ms/step - accuracy: 0.0629 - loss: 24.2557
Epoch 14/20
4303/4303 - 8s - 2ms/step - accuracy: 0.0648 - 

In [955]:
# Define list of stations names

stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'}

In [956]:
# Define the Confusion Matrix
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [957]:
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Pred        BASEL  MADRID
True                     
BASEL        3679       3
BELGRADE     1091       1
BUDAPEST      214       0
DEBILT         82       0
DUSSELDORF     29       0
HEATHROW       81       1
KASSEL         11       0
LJUBLJANA      61       0
MAASTRICHT      9       0
MADRID        445      13
MUNCHENB        7       1
OSLO            4       1
STOCKHOLM       4       0
VALENTIA        1       0


### 6.5. Configuration 5

In [958]:
# epochs = 20
# batch_size = 8
# n_hidden = 4
# activation='sigmoid'

epochs = 20
batch_size = 8
n_hidden = 4

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
# model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Input(shape=(timesteps, input_dim)))
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu'))
model.add(MaxPooling1D())
model.add(LSTM(n_hidden))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='sigmoid')) # Don't use relu here!

In [959]:
model.summary()

Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_14 (Conv1D)              │ (None, 14, 4)          │            76 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_14 (MaxPooling1D) │ (None, 7, 4)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_13 (LSTM)                  │ (None, 4)              │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 4)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 15)             │            75 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 295 (1.15 KB)

 Trainable params: 295 (1.15 KB)

 Non-trainable params: 0 (0.00 B)

In [960]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [961]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/20
2152/2152 - 7s - 3ms/step - accuracy: 0.1195 - loss: 8.8450
Epoch 2/20
2152/2152 - 5s - 2ms/step - accuracy: 0.1424 - loss: 9.0108
Epoch 3/20
2152/2152 - 5s - 2ms/step - accuracy: 0.1264 - loss: 9.3004
Epoch 4/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0920 - loss: 9.6385
Epoch 5/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0650 - loss: 10.0720
Epoch 6/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0663 - loss: 10.4512
Epoch 7/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0688 - loss: 10.8617
Epoch 8/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0676 - loss: 11.2997
Epoch 9/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0682 - loss: 11.6120
Epoch 10/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0683 - loss: 11.9980
Epoch 11/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0683 - loss: 12.2851
Epoch 12/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0681 - loss: 12.6432
Epoch 13/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0680 - loss: 12.9115
Epoch 14/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0679 - loss:

In [962]:
# Define list of stations names

stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'}

In [963]:
# Define the Confusion Matrix
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [964]:
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Pred        BASEL  BELGRADE  MADRID
True                               
BASEL        1446         1    2235
BELGRADE      988         0     104
BUDAPEST      207         0       7
DEBILT         82         0       0
DUSSELDORF     27         0       2
HEATHROW       70         0      12
KASSEL         11         0       0
LJUBLJANA      45         0      16
MAASTRICHT      3         0       6
MADRID        173         1     284
MUNCHENB        4         0       4
OSLO            4         0       1
STOCKHOLM       4         0       0
VALENTIA        0         0       1


### 6.6. Configuration 6

In [973]:
# epochs = 20
# batch_size = 16
# n_hidden = 4
# activation='sigmoid'

epochs = 20
batch_size = 16
n_hidden = 4

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
# model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Input(shape=(timesteps, input_dim)))
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu'))
model.add(MaxPooling1D())
model.add(LSTM(n_hidden))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='sigmoid')) # Don't use relu here!

In [974]:
model.summary()

Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_15 (Conv1D)              │ (None, 14, 4)          │            76 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_15 (MaxPooling1D) │ (None, 7, 4)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_14 (LSTM)                  │ (None, 4)              │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 4)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 15)             │            75 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 295 (1.15 KB)

 Trainable params: 295 (1.15 KB)

 Non-trainable params: 0 (0.00 B)

In [975]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [976]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/20
1076/1076 - 5s - 4ms/step - accuracy: 0.0926 - loss: 9.0677
Epoch 2/20
1076/1076 - 2s - 2ms/step - accuracy: 0.1116 - loss: 8.9969
Epoch 3/20
1076/1076 - 3s - 2ms/step - accuracy: 0.1042 - loss: 8.9272
Epoch 4/20
1076/1076 - 3s - 2ms/step - accuracy: 0.1186 - loss: 8.9424
Epoch 5/20
1076/1076 - 2s - 2ms/step - accuracy: 0.1104 - loss: 8.9581
Epoch 6/20
1076/1076 - 2s - 2ms/step - accuracy: 0.1105 - loss: 9.0271
Epoch 7/20
1076/1076 - 3s - 2ms/step - accuracy: 0.1181 - loss: 9.1129
Epoch 8/20
1076/1076 - 3s - 2ms/step - accuracy: 0.1196 - loss: 9.1837
Epoch 9/20
1076/1076 - 3s - 2ms/step - accuracy: 0.1200 - loss: 9.2980
Epoch 10/20
1076/1076 - 3s - 2ms/step - accuracy: 0.1214 - loss: 9.3988
Epoch 11/20
1076/1076 - 3s - 2ms/step - accuracy: 0.1278 - loss: 9.5164
Epoch 12/20
1076/1076 - 3s - 2ms/step - accuracy: 0.1202 - loss: 9.6356
Epoch 13/20
1076/1076 - 3s - 2ms/step - accuracy: 0.0784 - loss: 9.7780
Epoch 14/20
1076/1076 - 2s - 2ms/step - accuracy: 0.0654 - loss: 9.8866
E

In [977]:
# Define list of stations names

stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'}

In [978]:
# Define the Confusion Matrix
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [979]:
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Pred        BASEL  MADRID
True                     
BASEL        1628    2054
BELGRADE     1062      30
BUDAPEST      214       0
DEBILT         82       0
DUSSELDORF     29       0
HEATHROW       82       0
KASSEL         11       0
LJUBLJANA      61       0
MAASTRICHT      9       0
MADRID        284     174
MUNCHENB        8       0
OSLO            5       0
STOCKHOLM       4       0
VALENTIA        1       0


### 6.7. Configuration 7

In [1007]:
# epochs = 20
# batch_size = 8
# n_hidden = 8
# activation='sigmoid'

epochs = 20
batch_size = 8
n_hidden = 8

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
# model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Input(shape=(timesteps, input_dim)))
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu'))
model.add(MaxPooling1D())
model.add(LSTM(n_hidden))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='sigmoid')) # Don't use relu here!

In [1009]:
model.summary()

Model: "sequential_18"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_17 (Conv1D)              │ (None, 14, 8)          │           152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_17 (MaxPooling1D) │ (None, 7, 8)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_16 (LSTM)                  │ (None, 8)              │           544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 15)             │           135 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 831 (3.25 KB)

 Trainable params: 831 (3.25 KB)

 Non-trainable params: 0 (0.00 B)

In [1011]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [1013]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/20
2152/2152 - 7s - 3ms/step - accuracy: 0.0611 - loss: 9.2264
Epoch 2/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0823 - loss: 9.2486
Epoch 3/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0900 - loss: 9.5643
Epoch 4/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0891 - loss: 9.9306
Epoch 5/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0824 - loss: 10.2778
Epoch 6/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0705 - loss: 10.7353
Epoch 7/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0620 - loss: 11.1220
Epoch 8/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0581 - loss: 11.5364
Epoch 9/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0559 - loss: 11.9278
Epoch 10/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0546 - loss: 12.3894
Epoch 11/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0538 - loss: 12.7739
Epoch 12/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0539 - loss: 13.1589
Epoch 13/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0522 - loss: 13.5590
Epoch 14/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0550 - loss:

In [1014]:
# Define list of stations names

stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'}

In [1015]:
# Define the Confusion Matrix
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [1016]:
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Pred        BASEL  BUDAPEST  MADRID
True                               
BASEL        3668         1      13
BELGRADE     1083         0       9
BUDAPEST      213         0       1
DEBILT         82         0       0
DUSSELDORF     29         0       0
HEATHROW       81         0       1
KASSEL         11         0       0
LJUBLJANA      61         0       0
MAASTRICHT      9         0       0
MADRID        441         2      15
MUNCHENB        7         0       1
OSLO            5         0       0
STOCKHOLM       4         0       0
VALENTIA        1         0       0


### 6.8. Configuration 8

In [1017]:
# epochs = 20
# batch_size = 8
# n_hidden = 16
# activation='sigmoid'

epochs = 20
batch_size = 8
n_hidden = 16

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
# model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Input(shape=(timesteps, input_dim)))
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu'))
model.add(MaxPooling1D())
model.add(LSTM(n_hidden))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='sigmoid')) # Don't use relu here!

In [1018]:
model.summary()

Model: "sequential_19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_18 (Conv1D)              │ (None, 14, 16)         │           304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_18 (MaxPooling1D) │ (None, 7, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_17 (LSTM)                  │ (None, 16)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 15)             │           255 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,671 (10.43 KB)

 Trainable params: 2,671 (10.43 KB)

 Non-trainable params: 0 (0.00 B)

In [1019]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [1020]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/20
2152/2152 - 7s - 3ms/step - accuracy: 0.1278 - loss: 9.8473
Epoch 2/20
2152/2152 - 5s - 2ms/step - accuracy: 0.1012 - loss: 10.3320
Epoch 3/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0889 - loss: 10.9750
Epoch 4/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0788 - loss: 11.8086
Epoch 5/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0710 - loss: 12.5327
Epoch 6/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0634 - loss: 13.2988
Epoch 7/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0633 - loss: 13.8900
Epoch 8/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0575 - loss: 14.7844
Epoch 9/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0546 - loss: 15.4956
Epoch 10/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0568 - loss: 16.1481
Epoch 11/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0542 - loss: 16.8075
Epoch 12/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0518 - loss: 17.3622
Epoch 13/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0544 - loss: 17.9240
Epoch 14/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0522 - lo

In [1021]:
# Define list of stations names

stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'}

In [1022]:
# Define the Confusion Matrix
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [1023]:
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Pred        BASEL  BELGRADE  BUDAPEST  MADRID
True                                         
BASEL        3679         0         0       3
BELGRADE     1087         0         1       4
BUDAPEST      211         0         1       2
DEBILT         82         0         0       0
DUSSELDORF     29         0         0       0
HEATHROW       82         0         0       0
KASSEL         11         0         0       0
LJUBLJANA      60         0         0       1
MAASTRICHT      9         0         0       0
MADRID        450         1         0       7
MUNCHENB        7         0         0       1
OSLO            5         0         0       0
STOCKHOLM       4         0         0       0
VALENTIA        1         0         0       0


### 6.9. Configuration 9

In [1036]:
# epochs = 20
# batch_size = 8
# n_hidden = 32
# activation='sigmoid'

epochs = 20
batch_size = 8
n_hidden = 32

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
# model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Input(shape=(timesteps, input_dim)))
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu'))
model.add(MaxPooling1D())
model.add(LSTM(n_hidden))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='sigmoid')) # Don't use relu here!

In [1038]:
model.summary()

Model: "sequential_20"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_19 (Conv1D)              │ (None, 14, 32)         │           608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_19 (MaxPooling1D) │ (None, 7, 32)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_18 (LSTM)                  │ (None, 32)             │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 15)             │           495 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,423 (36.81 KB)

 Trainable params: 9,423 (36.81 KB)

 Non-trainable params: 0 (0.00 B)

In [1040]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [1042]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/20
2152/2152 - 8s - 4ms/step - accuracy: 0.0940 - loss: 10.5692
Epoch 2/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0942 - loss: 11.4242
Epoch 3/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0931 - loss: 12.4101
Epoch 4/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0874 - loss: 13.3871
Epoch 5/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0792 - loss: 14.6668
Epoch 6/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0704 - loss: 15.8908
Epoch 7/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0640 - loss: 17.1104
Epoch 8/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0682 - loss: 18.2676
Epoch 9/20
2152/2152 - 5s - 3ms/step - accuracy: 0.0639 - loss: 19.5528
Epoch 10/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0629 - loss: 20.8153
Epoch 11/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0599 - loss: 21.7489
Epoch 12/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0567 - loss: 22.7250
Epoch 13/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0578 - loss: 23.9151
Epoch 14/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0580 - l

In [1043]:
# Define list of stations names

stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'}

In [1044]:
# Define the Confusion Matrix
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [1045]:
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Pred        BASEL  MADRID
True                     
BASEL        3681       1
BELGRADE     1090       2
BUDAPEST      214       0
DEBILT         82       0
DUSSELDORF     29       0
HEATHROW       81       1
KASSEL         11       0
LJUBLJANA      61       0
MAASTRICHT      9       0
MADRID        457       1
MUNCHENB        8       0
OSLO            5       0
STOCKHOLM       4       0
VALENTIA        1       0


### 6.10. Configuration 10

In [1047]:
# epochs = 20
# batch_size = 8
# n_hidden = 64
# activation='sigmoid'

epochs = 20
batch_size = 8
n_hidden = 64

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
# model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Input(shape=(timesteps, input_dim)))
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu'))
model.add(MaxPooling1D())
model.add(LSTM(n_hidden))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='sigmoid')) # Don't use relu here!

In [1048]:
model.summary()

Model: "sequential_21"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_20 (Conv1D)              │ (None, 14, 64)         │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_20 (MaxPooling1D) │ (None, 7, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_19 (LSTM)                  │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_19 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 15)             │           975 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 35,215 (137.56 KB)

 Trainable params: 35,215 (137.56 KB)

 Non-trainable params: 0 (0.00 B)

In [1049]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [1050]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/20
2152/2152 - 7s - 3ms/step - accuracy: 0.1053 - loss: 11.4656
Epoch 2/20
2152/2152 - 6s - 3ms/step - accuracy: 0.0969 - loss: 13.1997
Epoch 3/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0873 - loss: 15.4549
Epoch 4/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0788 - loss: 17.3503
Epoch 5/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0709 - loss: 19.2708
Epoch 6/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0655 - loss: 21.2685
Epoch 7/20
2152/2152 - 5s - 3ms/step - accuracy: 0.0634 - loss: 23.3861
Epoch 8/20
2152/2152 - 5s - 3ms/step - accuracy: 0.0596 - loss: 25.3368
Epoch 9/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0591 - loss: 27.6049
Epoch 10/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0614 - loss: 29.6216
Epoch 11/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0625 - loss: 31.8629
Epoch 12/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0622 - loss: 33.3584
Epoch 13/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0634 - loss: 35.4392
Epoch 14/20
2152/2152 - 5s - 2ms/step - accuracy: 0.0632 - l

In [1051]:
# Define list of stations names

stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'}

In [1052]:
# Define the Confusion Matrix
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [1053]:
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Pred        BASEL  BELGRADE  BUDAPEST  MADRID
True                                         
BASEL        3679         1         0       2
BELGRADE     1092         0         0       0
BUDAPEST      214         0         0       0
DEBILT         82         0         0       0
DUSSELDORF     29         0         0       0
HEATHROW       82         0         0       0
KASSEL         11         0         0       0
LJUBLJANA      61         0         0       0
MAASTRICHT      9         0         0       0
MADRID        455         0         2       1
MUNCHENB        8         0         0       0
OSLO            5         0         0       0
STOCKHOLM       4         0         0       0
VALENTIA        1         0         0       0


### 6.11. Configuration 11

In [1061]:
# epochs = 30
# batch_size = 16
# n_hidden = 64
# activation='sigmoid'

# Set up the model
epochs = 30
batch_size = 16
n_hidden = 64

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
# model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Input(shape=(timesteps, input_dim)))
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu'))
model.add(MaxPooling1D())
model.add(LSTM(n_hidden))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='sigmoid')) # Don't use relu here!

In [1062]:
model.summary()

Model: "sequential_22"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_21 (Conv1D)              │ (None, 14, 64)         │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_21 (MaxPooling1D) │ (None, 7, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_20 (LSTM)                  │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 15)             │           975 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 35,215 (137.56 KB)

 Trainable params: 35,215 (137.56 KB)

 Non-trainable params: 0 (0.00 B)

In [1063]:
# Compile and run the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [1064]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
1076/1076 - 5s - 4ms/step - accuracy: 0.0949 - loss: 10.8860
Epoch 2/30
1076/1076 - 3s - 2ms/step - accuracy: 0.0984 - loss: 11.4379
Epoch 3/30
1076/1076 - 3s - 2ms/step - accuracy: 0.0958 - loss: 11.9369
Epoch 4/30
1076/1076 - 3s - 2ms/step - accuracy: 0.0948 - loss: 12.5914
Epoch 5/30
1076/1076 - 3s - 2ms/step - accuracy: 0.0946 - loss: 13.2301
Epoch 6/30
1076/1076 - 3s - 3ms/step - accuracy: 0.0911 - loss: 14.2052
Epoch 7/30
1076/1076 - 3s - 2ms/step - accuracy: 0.0863 - loss: 14.8483
Epoch 8/30
1076/1076 - 3s - 2ms/step - accuracy: 0.0874 - loss: 15.7047
Epoch 9/30
1076/1076 - 3s - 2ms/step - accuracy: 0.0855 - loss: 16.3487
Epoch 10/30
1076/1076 - 3s - 2ms/step - accuracy: 0.0822 - loss: 17.1813
Epoch 11/30
1076/1076 - 3s - 2ms/step - accuracy: 0.0776 - loss: 17.8635
Epoch 12/30
1076/1076 - 3s - 3ms/step - accuracy: 0.0756 - loss: 19.0705
Epoch 13/30
1076/1076 - 3s - 3ms/step - accuracy: 0.0746 - loss: 19.5045
Epoch 14/30
1076/1076 - 3s - 3ms/step - accuracy: 0.0744 - l

In [1065]:
# Create the Confusion Matrix

# Define list of stations names

stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'}

In [1066]:
# Define the Confusion Matrix
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [1067]:
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Pred        BASEL  MADRID
True                     
BASEL        3681       1
BELGRADE     1092       0
BUDAPEST      214       0
DEBILT         82       0
DUSSELDORF     29       0
HEATHROW       82       0
KASSEL         11       0
LJUBLJANA      61       0
MAASTRICHT      9       0
MADRID        458       0
MUNCHENB        8       0
OSLO            5       0
STOCKHOLM       4       0
VALENTIA        1       0


### 6.12. Configuration 12

In [1083]:
# epochs = 30
# batch_size = 16
# n_hidden = 64
# activation='tanh'

# Set up the model
epochs = 30
batch_size = 16
n_hidden = 64

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
# model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Input(shape=(timesteps, input_dim)))
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu'))
model.add(MaxPooling1D())
model.add(LSTM(n_hidden))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='tanh')) # Don't use relu here!

In [1085]:
model.summary()

Model: "sequential_23"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_22 (Conv1D)              │ (None, 14, 64)         │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_22 (MaxPooling1D) │ (None, 7, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_21 (LSTM)                  │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_21 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 15)             │           975 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 35,215 (137.56 KB)

 Trainable params: 35,215 (137.56 KB)

 Non-trainable params: 0 (0.00 B)

In [1087]:
# Compile and run the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [1089]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
1076/1076 - 6s - 5ms/step - accuracy: 0.1018 - loss: 24.4490
Epoch 2/30
1076/1076 - 3s - 3ms/step - accuracy: 0.0788 - loss: 24.7469
Epoch 3/30
1076/1076 - 3s - 3ms/step - accuracy: 0.0713 - loss: 24.8844
Epoch 4/30
1076/1076 - 3s - 3ms/step - accuracy: 0.0574 - loss: 24.6855
Epoch 5/30
1076/1076 - 3s - 3ms/step - accuracy: 0.0600 - loss: 24.7778
Epoch 6/30
1076/1076 - 3s - 3ms/step - accuracy: 0.0570 - loss: 24.4695
Epoch 7/30
1076/1076 - 3s - 3ms/step - accuracy: 0.0732 - loss: 24.5180
Epoch 8/30
1076/1076 - 3s - 3ms/step - accuracy: 0.0834 - loss: 24.8120
Epoch 9/30
1076/1076 - 3s - 3ms/step - accuracy: 0.0602 - loss: 25.3291
Epoch 10/30
1076/1076 - 3s - 3ms/step - accuracy: 0.0196 - loss: 24.3301
Epoch 11/30
1076/1076 - 3s - 3ms/step - accuracy: 0.0535 - loss: 24.7521
Epoch 12/30
1076/1076 - 3s - 3ms/step - accuracy: 0.0748 - loss: 25.1372
Epoch 13/30
1076/1076 - 3s - 3ms/step - accuracy: 0.0359 - loss: 24.5783
Epoch 14/30
1076/1076 - 3s - 3ms/step - accuracy: 0.0616 - l

In [1090]:
# Create the Confusion Matrix

# Define list of stations names

stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'}

In [1091]:
# Define the Confusion Matrix
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [1092]:
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Pred        LJUBLJANA  MADRID  MUNCHENB  VALENTIA
True                                             
BASEL             288    2004       249      1141
BELGRADE            0     978        16        98
BUDAPEST            0     201         0        13
DEBILT              0      82         0         0
DUSSELDORF          0      28         1         0
HEATHROW            0      75         1         6
KASSEL              0      11         0         0
LJUBLJANA           0      46         0        15
MAASTRICHT          0       8         0         1
MADRID              3     240        34       181
MUNCHENB            0       7         1         0
OSLO                0       5         0         0
STOCKHOLM           0       4         0         0
VALENTIA            0       1         0         0


## 7. Create and configure CNN model

### 7.1 Configuration 1

In [1134]:
# epochs = 10
# batch_size = 4
# n_hidden = 4
# activation='softmax'

epochs = 10
batch_size = 4
n_hidden = 4

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
# model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Input(shape=(timesteps, input_dim)))
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) # Options: sigmoid, tanh, softmax, relu

In [1136]:
model.summary()

Model: "sequential_27"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_26 (Conv1D)              │ (None, 14, 4)          │            76 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 14, 16)         │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_26 (MaxPooling1D) │ (None, 7, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 15)             │         1,695 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,851 (7.23 KB)

 Trainable params: 1,851 (7.23 KB)

 Non-trainable params: 0 (0.00 B)

In [1138]:
# Compile and run the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [1140]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/10
4303/4303 - 6s - 1ms/step - accuracy: 0.1274 - loss: 23539.0273
Epoch 2/10
4303/4303 - 5s - 1ms/step - accuracy: 0.1301 - loss: 207403.5000
Epoch 3/10
4303/4303 - 5s - 1ms/step - accuracy: 0.1261 - loss: 689949.8125
Epoch 4/10
4303/4303 - 5s - 1ms/step - accuracy: 0.1283 - loss: 1509194.7500
Epoch 5/10
4303/4303 - 5s - 1ms/step - accuracy: 0.1324 - loss: 2812083.2500
Epoch 6/10
4303/4303 - 6s - 1ms/step - accuracy: 0.1305 - loss: 4738468.0000
Epoch 7/10
4303/4303 - 5s - 1ms/step - accuracy: 0.1333 - loss: 7318947.5000
Epoch 8/10
4303/4303 - 5s - 1ms/step - accuracy: 0.1299 - loss: 10591554.0000
Epoch 9/10
4303/4303 - 5s - 1ms/step - accuracy: 0.1344 - loss: 14892505.0000
Epoch 10/10
4303/4303 - 5s - 1ms/step - accuracy: 0.1275 - loss: 20030088.0000


In [1142]:
# Create the Confusion Matrix

# Define list of stations names

stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'}

In [1144]:
# Define the Confusion Matrix
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [1146]:
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  DUSSELDORF  HEATHROW  KASSEL  \
True                                                                          
BASEL         170         1       536     110         310        57    1022   
BELGRADE        0         0       124       0          50         1     339   
BUDAPEST        0         0        24       0           6         0      54   
DEBILT          0         0         9       1           6         1      20   
DUSSELDORF      0         0         2       0           3         1       5   
HEATHROW        0         0         1       0           2         1      14   
KASSEL          0         0         0       0           3         0       2   
LJUBLJANA       0         0         8       0           1         0       5   
MAASTRICHT      0         0         1       0           1         0       4   
MADRID          5         0         3       6           6         4      48   
MUNCHENB 

### 7.2. Configuration 2

In [1148]:
# epochs = 15
# batch_size = 8
# n_hidden = 8
# activation='softmax'

epochs = 15
batch_size = 8
n_hidden = 8

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
# model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Input(shape=(timesteps, input_dim)))
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) # Options: sigmoid, tanh, softmax, relu

In [1150]:
model.summary()

Model: "sequential_28"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_27 (Conv1D)              │ (None, 14, 8)          │           152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (None, 14, 16)         │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_27 (MaxPooling1D) │ (None, 7, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 15)             │         1,695 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,991 (7.78 KB)

 Trainable params: 1,991 (7.78 KB)

 Non-trainable params: 0 (0.00 B)

In [1152]:
# Compile and run the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [1154]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/15
2152/2152 - 4s - 2ms/step - accuracy: 0.1282 - loss: 8605.3682
Epoch 2/15
2152/2152 - 3s - 1ms/step - accuracy: 0.1335 - loss: 87945.9766
Epoch 3/15
2152/2152 - 3s - 1ms/step - accuracy: 0.1317 - loss: 266854.8125
Epoch 4/15
2152/2152 - 3s - 1ms/step - accuracy: 0.1349 - loss: 588459.3125
Epoch 5/15
2152/2152 - 3s - 1ms/step - accuracy: 0.1325 - loss: 1051891.6250
Epoch 6/15
2152/2152 - 3s - 1ms/step - accuracy: 0.1331 - loss: 1714182.6250
Epoch 7/15
2152/2152 - 3s - 1ms/step - accuracy: 0.1348 - loss: 2526734.7500
Epoch 8/15
2152/2152 - 3s - 1ms/step - accuracy: 0.1320 - loss: 3635063.2500
Epoch 9/15
2152/2152 - 3s - 1ms/step - accuracy: 0.1325 - loss: 4957855.5000
Epoch 10/15
2152/2152 - 3s - 1ms/step - accuracy: 0.1279 - loss: 6606738.5000
Epoch 11/15
2152/2152 - 3s - 1ms/step - accuracy: 0.1323 - loss: 8552280.0000
Epoch 12/15
2152/2152 - 3s - 1ms/step - accuracy: 0.1311 - loss: 10796995.0000
Epoch 13/15
2152/2152 - 3s - 1ms/step - accuracy: 0.1304 - loss: 13447819.0000


In [1156]:
# Create the Confusion Matrix

# Define list of stations names

stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'}

In [1158]:
# Define the Confusion Matrix
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [1160]:
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  HEATHROW  KASSEL  LJUBLJANA  \
True                                                                         
BASEL         478        16      1230      44       775     504        319   
BELGRADE        1         1       771       0       254      46          6   
BUDAPEST        0         0       131       0        71       8          0   
DEBILT          0         0        33       0        42       5          0   
DUSSELDORF      0         0         9       0        19       1          0   
HEATHROW        0         0        21       0        58       3          0   
KASSEL          0         0         7       0         1       2          0   
LJUBLJANA       0         0        38       0        14       0          3   
MAASTRICHT      0         0         4       0         2       0          3   
MADRID         11         0       135       0       173      44         49   
MUNCHENB        0      

### 7.3. Configuration 3

In [1162]:
# epochs = 20
# batch_size = 4
# n_hidden = 64
# activation='softmax'

epochs = 20
batch_size = 4
n_hidden = 64

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
# model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Input(shape=(timesteps, input_dim)))
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) # Options: sigmoid, tanh, softmax, relu

In [1164]:
model.summary()

Model: "sequential_29"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_28 (Conv1D)              │ (None, 14, 64)         │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_32 (Dense)                │ (None, 14, 16)         │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_28 (MaxPooling1D) │ (None, 7, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_33 (Dense)                │ (None, 15)             │         1,695 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,951 (15.43 KB)

 Trainable params: 3,951 (15.43 KB)

 Non-trainable params: 0 (0.00 B)

In [1166]:
# Compile and run the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [1168]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/20
4303/4303 - 7s - 2ms/step - accuracy: 0.1299 - loss: 315509.6250
Epoch 2/20
4303/4303 - 6s - 1ms/step - accuracy: 0.1344 - loss: 2977205.5000
Epoch 3/20
4303/4303 - 6s - 1ms/step - accuracy: 0.1358 - loss: 9606317.0000
Epoch 4/20
4303/4303 - 6s - 1ms/step - accuracy: 0.1376 - loss: 21973630.0000
Epoch 5/20
4303/4303 - 6s - 1ms/step - accuracy: 0.1339 - loss: 41310788.0000
Epoch 6/20
4303/4303 - 6s - 1ms/step - accuracy: 0.1333 - loss: 68338352.0000
Epoch 7/20
4303/4303 - 6s - 1ms/step - accuracy: 0.1314 - loss: 107062816.0000
Epoch 8/20
4303/4303 - 6s - 1ms/step - accuracy: 0.1309 - loss: 154237264.0000
Epoch 9/20
4303/4303 - 6s - 1ms/step - accuracy: 0.1293 - loss: 215491056.0000
Epoch 10/20
4303/4303 - 6s - 1ms/step - accuracy: 0.1286 - loss: 291783744.0000
Epoch 11/20
4303/4303 - 5s - 1ms/step - accuracy: 0.1300 - loss: 380743872.0000
Epoch 12/20
4303/4303 - 6s - 1ms/step - accuracy: 0.1253 - loss: 487769280.0000
Epoch 13/20
4303/4303 - 6s - 1ms/step - accuracy: 0.1298 - 

In [1170]:
# Create the Confusion Matrix

# Define list of stations names

stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'}

In [1172]:
# Define the Confusion Matrix
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [1092]:
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Pred        LJUBLJANA  MADRID  MUNCHENB  VALENTIA
True                                             
BASEL             288    2004       249      1141
BELGRADE            0     978        16        98
BUDAPEST            0     201         0        13
DEBILT              0      82         0         0
DUSSELDORF          0      28         1         0
HEATHROW            0      75         1         6
KASSEL              0      11         0         0
LJUBLJANA           0      46         0        15
MAASTRICHT          0       8         0         1
MADRID              3     240        34       181
MUNCHENB            0       7         1         0
OSLO                0       5         0         0
STOCKHOLM           0       4         0         0
VALENTIA            0       1         0         0
